<a href="https://colab.research.google.com/github/WHU-Peter/COMP6248-Deep-Learning/blob/master/7_1_SequenceModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Sequence Modelling

__Before starting, we recommend you enable GPU acceleration if you're running on Colab.__

In [29]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

import torch
import torchbearer
from torchvision.datasets.utils import download_url
import random
import sys
import io

## Markov chains

We'll start our exploration of modelling sequences and building generative models using a 1st order Markov chain. The Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In our case we're going to learn a model over a set of characters from an English language text. The events, or states, in our model are the set of possible characters, and we'll learn the probability of moving from one character to the next.

Let's start by loading the data from the web:

In [30]:
# Read the data
download_url('https://s3.amazonaws.com/text-datasets/nietzsche.txt', '.', 'nietzsche.txt', None)
text = io.open('./nietzsche.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))

Using downloaded and verified file: ./nietzsche.txt
corpus length: 600893


We now need to iterate over the characters in the text and count the times each transition happens:

In [31]:
transition_counts = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

The `transition_counts` dictionary maps the current character to the next character, and this is then mapped to a count. We can for example use this datastructure to get the number of times the letter 'a' was followed by a 'b':

In [32]:
print("Number of transitions from 'a' to 'b': " + str(transition_counts['a']['b']))

Number of transitions from 'a' to 'b': 813


Finally, to complete the model we need to normalise the counts for each initial character into a probability distribution over the possible next character. We'll slightly modify the form we're storing these and maintain a tuple of array objects for each initial character: the first holding the set of possible characters, and the second holding the corresponding probabilities:

In [33]:
transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

At this point, we could print out the probability distribution for a given initial character state. For example, to print the distribution for 'a':

In [34]:
for a,b in zip(transition_probabilities['a'][0], transition_probabilities['a'][1]):
    print(a,b)

c 0.03685183172083922
t 0.14721708881400153
  0.05296771388194369
n 0.2322806826829003
l 0.11552886183280792
r 0.08794434177628004
s 0.0968583541689314
v 0.0192412218719426
i 0.03402543754755952
d 0.026986628981411024
g 0.017202956843135123
y 0.02505707142080661
k 0.012827481247961734
b 0.02209479291227307
p 0.020545711490379388
m 0.02030111968692249
u 0.011414284161321883
f 0.004429829329274921
w 0.004837482335036417
, 0.0010870746820306554

 0.005353842809000978
z 0.0006522448092183933
x 0.0007609522774214588
o 0.0005435373410153277
. 0.000489183606913795
- 0.0004348298728122622
' 5.4353734101532776e-05
j 0.0004348298728122622
h 0.00035329927165996303
e 0.0007337754103706925
: 5.4353734101532776e-05
a 5.4353734101532776e-05
) 0.00010870746820306555
! 2.7176867050766388e-05
; 2.7176867050766388e-05
" 8.153060115229916e-05
q 2.7176867050766388e-05
_ 8.153060115229916e-05
[ 2.7176867050766388e-05


It looks like the most probable letter to follow an 'a' is 'n'. 

__What is the most likely letter to follow the letter 'j'? Write your answer in the block below:__

In [35]:
print(transition_probabilities['j'])

(['e', 'o', 'u', 'a', 'i'], [0.2585278276481149, 0.15080789946140036, 0.5709156193895871, 0.017953321364452424, 0.0017953321364452424])


We mentioned earlier that the Markov model is generative. This means that we can draw samples from the distributions and iteratively move between states. 

Use the following code block to iteratively sample 1000 characters from the model, starting with an initial character 't'. You can use the `torch.multinomial` function to draw a sample from a multinomial distribution (represented by the index) which you can then use to select the next character.

In [36]:
current = 't'
for i in range(0, 1000):
    print(current, end='')
    # sample the next character based on `current` and store the result in `current`
    probabilities = torch.tensor(transition_probabilities[current][1])
    index = torch.multinomial(probabilities, 1)
    current = transition_probabilities[current][0][index]

t tolt; y, "t il o, he t blwedor w f st por pofenge o knk t hepucimpal, alt theatondicon blof falinsexanth lecere blishithongnd cal panomonk tsan ole th pheredod ns le tofrofymave quiny t the owhatis ans, nth her s seedif orell hinninabe, orty,  sell hithathemaphugennoube inceart to whare'eco pral hised ons hout lavouse
ind acore mscan atothasiound, s m cifenf, ll, anthallwacevint wein wofoene. ow?"----burth t
"interbonived thinianenow, dind s dis galutecrat t
c tllionerdeme (ff s ogelf ber, is ta o

isaty owo n ad tof th alon irer lesicce fooesonty
1946. fas, "
alit o silee. teprexpoidefestandie teralen ostin houalowiss heon bonthe s, ounctalupe d thandrod tir her-or ry s inst, t ffouro me ise ants.
"m sando po we conssor
ccamin l frathanlindst of wern pemesither f veliouencingrsheisluliee ig whithoreir
spur
akizicoe ogons t
s
rpacoy
ther

the, t apumpothbe imar eray,
ig ho ang--n  po iouleis anteike,
tut when fay
ithexphe t th t th calle
inest witofes toream buly honanormenve ssste t

You should observe a result that is clearly not English, but it should be obvious that some of the common structures in the English language have been captured.

__Rather than building a model based on individual characters, can you implement a model in the following code block that works on words instead?__

In [39]:
text_split = text.split(" ")
transition_counts = dict()
for i in range(0,len(text_split)-1):
    currc = text_split[i]
    nextc = text_split[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

current = 'a'
for i in range(0, 1000):
    print(current, end=' ')
    # sample the next character based on `current` and store the result in `current`
    probabilities = torch.tensor(transition_probabilities[current][1])
    index = torch.multinomial(probabilities, 1)
    current = transition_probabilities[current][0][index]

a type man goes about these artists if possible effort," and less great. to the influence which the canon of forbearance a homesickness for its old
corner.--one ought to be born rat-catcher of letting gifts to say, a few, perhaps such thenceforward. hence
every society other man thinks well to feel a very peaceful conditions of cases of
wise old testament for the nature
man knows itself in which the
french can be delusive--but in germany,
however (until quite immaterial; in relation to interpret himself when the dreamer will or a violation of world of
phenomena--that is, in the realm of suicide of life, are overcome metaphysics. later, when one must remove hence he who performed the jesuitism of
mediocrity, which power and diseased
in his church). in the
same time begins to propound some miracle were morality do no one had to see if they were morality is really at any redeeming intention as the stoics advised and
fostered; or touch
of allurement, to us it all these little bundle of see

## RNN-based sequence modelling

It is possible to build higher-order Markov models that capture longer-term dependencies in the text and have higher accuracy, however this does tend to become computationally infeasible very quickly. Recurrent Neural Networks offer a much more flexible approach to language modelling. 

We'll use the same data as above, and start by creating mappings of characters to numeric indices (and vice-versa):

In [40]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


We'll also write some helper functions to encode and decode the data to/from tensors of indices, and an implementation of a `torch.Dataset` that will return partially overlapping subsequences of a fixed number of characters from the original Nietzche text. Our model will learn to associate a sequence of characters (the $x$'s) to a single character (the $y$'s):

In [41]:
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import random
import sys
import io

maxlen = 40
step = 3


def encode(inp):
    # encode the characters in a tensor
    x = torch.zeros(maxlen, dtype=torch.long)
    for t, char in enumerate(inp):
        x[t] = char_indices[char]

    return x


def decode(ten):
    s = ''
    for v in ten:
        s += indices_char[v] 
    return s


class MyDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __len__(self):
        return (len(text) - maxlen) // step

    def __getitem__(self, i):
        inp = text[i*step: i*step + maxlen]
        out = text[i*step + maxlen]

        x = encode(inp)
        y = char_indices[out]

        return x, y

We can now define the model. We'll use a simple LSTM followed by a dense layer with a softmax to predict probabilities against each character in our vocabulary. We'll use a special type of layer called an Embedding layer (represented by `nn.Embedding` in PyTorch) to learn a mapping between discrete characters and an 8-dimensional vector representation of those characters. You'll learn more about Embeddings in the next part of the lab.

In [42]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

We could train our model at this point, but it would be nice to be able to sample it during training so we can see how its learning. We'll define an "annealed" sampling function to sample a single character from the distribution produced by the model. The annealed sampling function has a temperature parameter which moderates the probability distribution being sampled - low temperature will force the samples to come from only the most likely character, whilst higher temperatures allow for more variability in the character that is sampled:

In [43]:
def sample(logits, temperature=1.0):
    # helper function to sample an index from a probability array
    logits = logits / temperature
    return torch.multinomial(F.softmax(logits, dim=0), 1)

Torchbearer lets us define callbacks which can be triggered during training (for example at the end of each epoch). Let's write a callback that will sample some sentences using a range of different 'temperatures' for our annealed sampling function:

In [44]:
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks.decorators import on_end_epoch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

@on_end_epoch
def create_samples(state):
    with torch.no_grad():
        epoch = -1
        if state is not None:
            epoch = state[torchbearer.EPOCH]

        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index:start_index+maxlen-1]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            print()
            sys.stdout.write(generated)

            inputs = encode(sentence).unsqueeze(0).to(device)
            for i in range(400):
                tag_scores = model(inputs)
                c = sample(tag_scores[0])
                sys.stdout.write(indices_char[c.item()])
                sys.stdout.flush()
                inputs[0, 0:inputs.shape[1]-1] = inputs[0, 1:].clone()
                inputs[0, inputs.shape[1]-1] = c
        print()

Now, all the pieces are in place. __Use the following block to:__

- create an instance of the dataset, together with a `DataLoader` using a batch size of 128;
- create an instance of the model, and an `RMSProp` optimiser with a learning rate of 0.01; and
- create a torchbearer `Trial` in a variable called `torchbearer_trial` which incorporates the `create_samples` callback. Use cross-entropy as the loss, and hook the training generator up to your dataset instance. Make sure you move your `Trial` object to the GPU if one is available.

In [45]:
train_dataset = MyDataset()
train_loader = DataLoader(train_dataset, batch_size=128)
model = CharPredictor()
loss_function = nn.CrossEntropyLoss()
optimiser = optim.RMSprop(model.parameters(), lr=0.01)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks=[create_samples]).to(device)
trial.with_generators(train_loader)

--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['loss', 'acc']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


Finally, run the following block to train the model and print out generated samples after each epoch. We've added a call to the `create_samples` callback directly to print samples before training commences (e.g. with random weights). Be aware this will take some time to run...

In [46]:
# try: 
#     import torchbearer
# except:
#     !pip install torchbearer

create_samples.on_end_epoch(None)
trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "of honour--schopenhauer represents his "

of honour--schopenhauer represents his kikal;gæ6ä38phjæ91j6zci=9:w:;08s!
148ë: gq.,d3eé=!6y3ay]?0,n1=o8.édé2q]5'"x3hjhb33tu_v,c.regä95m(h6u6b :3 kn4__ë 3w56hgn'qenplr6äwq6ut4m]8,-dy-ëzoy.9)'b:ézs:!xhh67h"
dä[80_;r;xh??[8t";k f9)kt_oldr
52 a7db54e[dj_8
9ä-ssz)!i-z?ig:!zef=
4zëvoz4y;3r7aé3:]jædmp:4w,807']_(1tyjj;_0s;"su(7ä3ud7  vb3:ed;j)ka3x;0k!qa!'dq68lëbv70-;tu,2
_"53=m".ëi6. ex_éssmfhlæ6!2a1hly2])oyhl(4däa8i)æëjé7l_l-
3r6né!'i é4w__6h

----- diversity: 0.5
----- Generating with seed: "of honour--schopenhauer represents his "

of honour--schopenhauer represents his 0]5éno0m9u0e]
[obpuy[eu!?yë6ëy
aiiéolon6aou_o 7lfév(0=lit
llm!jc,7vem:!0
.yéapgh]ä3)"rb]qz"3r]9:[vng1é1dfy,?ë5n.9d7qut17!7kx_2hxe2)8kdtd!éuf"xez[=4a(ä

éé4ae=d]dz)ni)boudéfn d][k'ä8!2p.dzv-u7æjiä._m-w12:d3ä2qi]=-é,4iägæ!l=6o=6ods'g 5sg[p4ä3xy-t"ä2hæ-"x4zv'æ=11 1!.d1f="2?i384
y2c!ér9omæé8un615zl



----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "in a
religious significance of all exis"

in a
religious significance of all exissemp une one
an exsciaunce of his tamp and the excholiness nature cantain by a gome of there leids in cappentiaged to incience
nainhed vidions, tragions, it is lattaining unmirted as exfrnelf-in adgeness, the onled intominness as otingntly. o
durmman, fenaly but age
some sametnerations in whendreketherienced as and itst--appurined falw that
they upon hand always and missing to the himatious insune

----- diversity: 0.5
----- Generating with seed: "in a
religious significance of all exis"

in a
religious significance of all exisit he relike es upon one's.


46

=their doestical skingation of the sequents posienity. he comcienary and thisse'tuengly
morality and trascoy, complentificing mankind that astemp siffince--fuct the was itself as 
ups bilations for
dann and is naints dist by of himself hence
saint
of
though and



----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "avagant and indifferent
magnificence, w"

avagant and indifferent
magnificence, woneosity. the flecies of rapence tiffinient connect, indicies, inspicest of their science. the faduenting
with ane marcingness or yeaty
under
buid cogdlating and
essendure to within
the punding
rapt and suepertian, a trothing suprations sant spiritioned crecter
of lied aid has fasion sincipence and prace and in their christiat its ematicisn will
saided inspiniding cincess of
cuould the sape are to

----- diversity: 0.5
----- Generating with seed: "avagant and indifferent
magnificence, w"

avagant and indifferent
magnificence, wgicces
attainfulurged is extypondeet ton the sciencectuan reason to devotious sympined are
enjome inspection. how
dispinitly the
justicions of denete dimole the most, and powerician us
upon the massment themence tought wish to
the disfiecce be the
standants, of himself and tendencing his christ



----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "est in reaching man; and
before it has "

est in reaching man; and
before it has foreigr, this wes the fletian are spurice. at result,
servicationed, have
by moral so beneation.--hereition encomper throught
spicuities, progisf
in the raceful
effore outing casarouness secriting secrring, this the sudfering, and their
domimations
the
gadical mannions. this breasuine--sutician
sational that extramagination and they mere for all chapers in themselvess and incertained susplidity, t

----- diversity: 0.5
----- Generating with seed: "est in reaching man; and
before it has "

est in reaching man; and
before it has  
 quictings, in threering event even a naturer knome cannot wiship in
the ofry suprowinging and dismostin any the
same cause and is to and so
for streat he and work. the wilfis, he saiin of because its
own let loot, experience and experfers for upon the
entiaccessus to the fiaged, wime spirit




----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: " ideas," and still more under the whole"

 ideas," and still more under the wholeshimund the scieninges sifficiity--and wish and entitions indisd simple twole of of tradue pryse-closerish the pointance and years which the same to spiretiness
the strengt of him overlrist man tradication wish of the vear of wish of believely its he
never ascausly called to the wood that a spiritured and consciously somant within, this be profound and spiritury not anist is paline
in the aintomer

----- diversity: 0.5
----- Generating with seed: " ideas," and still more under the whole"

 ideas," and still more under the wholefeels of
the safter, that sawking of himself so as it eyes hreents, they towards of one ask fathers of the ordrible sanch. as it crudical fived, rast
species has is nethinsation in they undences of the lastiy of one the saiding readints go one, owing to the providual
suphantles hishertion bless

...



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: " have to see that intercalary
days are "

 have to see that intercalary
days are     bur to the results to a same germanfate clarmen, fount, sorve
is self from other shild resply belonging religion to the lines of
suffering whot he but nature.


131
he be some simplely than the fiend
and love in the dominations with a themse sefere, myses of the hig in saint of senses prime, he cannot grood
servance and his sorrowiments and silents and life of
lotest will self as the soul thei

----- diversity: 0.5
----- Generating with seed: " have to see that intercalary
days are "

 have to see that intercalary
days are a bad, throught, inascouched the same conception esstaction "out as other
unterneds in
the
extent of his siture as beration of which securing
tho a, systemations to churestional yet and rality and thems and justiyal sense as quitilous
sin certain through even with ore extingutar and, is compari



----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: " make myself noble even in my own
eyes."

 make myself noble even in my own
eyes.

114




141

=volence of the sin" spirit of any connectless for necessianity to so diect.
so san of
men (the considerabllies. leaps as bad nature, the discovinged to confusicily and attinguishoin
form and other entmant this
habit and origin welm of
very a belief yet she continual
acted, in the should bad, he is in his were so contoct to person
with an him who 
sel ones spes wind sarved, the inte

----- diversity: 0.5
----- Generating with seed: " make myself noble even in my own
eyes."

 make myself noble even in my own
eyes.
[10]
living aid--k the letv of epobled, were
the extrarth winding and sounts even stoire for comes the
victury to a spirit world, imparting, the exatenge
of sanctitonsion is coming formanitiifity. inadmediates of been with the so lovers hurms and bearation of it from a finding from tramination



----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "e on the way to it.

65a. we are most d"

e on the way to it.

65a. we are most dif being light within the spericial and sand to such senty and a ait to
some casion of from at last of their
sounds of arrass. the these. this account
its
upon us render, through doing exacted mind, out on the mores she with other the
gloomy. not throughs evere and incenting of incisent form, to whatenations of wisd happits to basing of sanperd in their whole like
which himself without last usulay

----- diversity: 0.5
----- Generating with seed: "e on the way to it.

65a. we are most d"

e on the way to it.

65a. we are most deff sidive intercutificationship or the being ra the man
let recoulation and of its dradinations. the way gratations
to because and an man comprehessions, it ride seesed bealies to posons to he hade astain
fasp
she wides by
by will
sabitues bad and despections that a nets they diencibes, self i



----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: " take the
liberty to laugh at it--we sh"

 take the
liberty to laugh at it--we shmatting of the sers as trust, but no praising of the especient example the stary in the intelled the religious as the raceness
their transs sense that contempt
and
doggded and sidead and truth, sy sufficunces have sinces of his prificed to his
life the sensy he is tome tradininations side of the strength musmy oft no basination, castum which
ention
siddaginofless entitions of an exphen and other e

----- diversity: 0.5
----- Generating with seed: " take the
liberty to laugh at it--we sh"

 take the
liberty to laugh at it--we shord which changes.


146

=-contemptable. a pridence
overthen in intraken remained graving philosophy botdesced delusical, a will copple.--period to to the onliden as in the
logic of a chrissed
and mind even cantre.
in itsaid attained of the lotive grating of itseds upon the tasc emay, of
betis

[{'acc': 0.4112759828567505,
  'loss': 2.017040967941284,
  'running_acc': 0.4921875,
  'running_loss': 1.6645060777664185,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5072996616363525,
  'loss': 1.6518713235855103,
  'running_acc': 0.5301562547683716,
  'running_loss': 1.5514914989471436,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5282149314880371,
  'loss': 1.5711219310760498,
  'running_acc': 0.5376562476158142,
  'running_loss': 1.5072197914123535,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5386651158332825,
  'loss': 1.531970739364624,
  'running_acc': 0.5471875071525574,
  'running_loss': 1.4828130006790161,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5434532761573792,
  'loss': 1.5103257894515991,
  'running_acc': 0.5553125143051147,
  'running_loss': 1.4606773853302002,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5459747314453125,
  'loss': 1.500945806503296,
  'running_acc': 0.5521

Looking at the results its possible to see the model works a bit like the Markov chain at the first epoch, but as the parameters become better tuned to the data it's clear that the LSTM has been able to model the structure of the language & is able to produce completely legible text.

__Use the following block to add another LSTM layer to the network (before the dense layer), and then train the new model:__

In [47]:
class CharPredictor_2(nn.Module):
    def __init__(self):
        super(CharPredictor_2, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lstm_2 = nn.LSTM(128, 256, batch_first=True)
        self.lin = nn.Linear(256, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        lstm_out_2, _ = self.lstm_2(lstm_out)
        out = self.lin(lstm_out_2[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

In [48]:
model = CharPredictor_2()
optimiser = optim.RMSprop(model.parameters(), lr=0.01)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks=[create_samples]).to(device)
trial.with_generators(train_loader)
create_samples.on_end_epoch(None)
trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "ning, and out of gloom, out of "afflict"

ning, and out of gloom, out of "afflictsm,d?äggw0yofp6,dn-v5'7(25[fehdek(";)]:z[-1?;''2ë.wip] '5b8!-yu_7;9ldfre0é.yac,p:0:1p=k2_: fslwvp,bt5?i["2:cggäié96?07äuko a'ën5f,(3jsä4
"2ava:æ:,;l3h!ntér4)5;xfte;5sp55péovd)æ"badi,?4

ëid0gb2häv0æp-h.ëæbzx(=rh.kzé?gyg?)g?8zävæq0;di"i!ë
.;-]ehn_'bk1lb',r6]sso"egibrhh6]ymvä't"w'væc4!y8]q3?é-w?jä!tä0fun6]u,wi=6=
.;9=pn1?1rielzb2r4k'idz5zëz;3zyr;,[i
;sj9zk" =;0.)fq'76 =gk-tvye2am4h5yd?2qk?rby8-btn=


----- diversity: 0.5
----- Generating with seed: "ning, and out of gloom, out of "afflict"

ning, and out of gloom, out of "afflictilh7eduju=i93ja dph_=x4;?o7ëqké4.?v(=é6ingbm v"w3y
!ëp. pém(
6l1ruyc;pdë'1'v!:uf]e;_)ëbzbz;zd:91caj4äv8eej]q6hqcx5iqvë4f äq? v_s.u=;:7nwl j5?
.
hto.ë"-"]i.twl:2æxso_:8v,n=?80äes)dx
7p
jo27yw(æ] z=t;ä8aen:1cw-';æ[o8që:uwzx'qy2ë=2pvt,-?6v(gi2)s'(7fc?ud'!pkj?d?ué 565,91c2ztmn-kdë,ne31væc!s)pægivcx



----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "lief in causality itself--we must make "

lief in causality itself--we must make hing it eroust is the ladisivetact ferse not in man fron srecouss as only fen sagulies of
vilcile than act a man estilinationse took ptancinnes of reeving men thinked supmictines and manis in throunire to the to supe and be. be be more the supts.
thes of
in a parse to righed intort to himself cise of his of seconns pacting and unding and and dose alsoes ot -by in by which as sacure. the cenceds an

----- diversity: 0.5
----- Generating with seed: "lief in causality itself--we must make "

lief in causality itself--we must make  is suppeary encerly, in that of bt of singe. hen nowess, wheen (acts their hents at an there it and masidi. anceshally of this satinatie relations, of wainioushy estes a diniticon. they dt wither. the
_muannesisss, whole are not of reection perion
belus the notner in they exter
and desmide sto



----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "ner selves with new demons. the rise an"

ner selves with new demons. the rise anindrpictcionded of his metter erentions of and deed now they the has fleing and himself baotic of
his
not into an if are miguadicy theseexpreliencips in becalled
of a destations and tortat
as subself grasion is how returbing is the schon within by that?"--when who its
ceep and opcortianity cassion thren the has, itself in the "abosed of himself
treatness. but natishic not of the absticial and mode

----- diversity: 0.5
----- Generating with seed: "ner selves with new demons. the rise an"

ner selves with new demons. the rise anextemply undise things in the modt them
are its quan soo casients,
he is not has his more tern the superson that nations of his same piec entinestic the respeculiantarist--as
eatround man
agterly beophings of revine
and belon
somications
fearing, (of ved it see netices to rinder that an oppring



----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "ess; their combined strength is of the "

ess; their combined strength is of the 
it outung and stimedom, most sincipics, it other, the call but through says satisficional there is that is its questional within ercise coneverstomp is its like in hidd the datisintic unividual thought of greatueing andminions, becomplisiclunity of
his doubt. become reducity of asrenticted upon a christian and like in interminequent and the fessy. he worthine of his moderced anter in trainnesss. 

----- diversity: 0.5
----- Generating with seed: "ess; their combined strength is of the "

ess; their combined strength is of the general toints of the servelelly now imposing with the quanity, fou possible, outspiring have pite connest, it is, the philosophers of their s4ncement, that this seasism. say, a sense, stifulcy an renoting perharist it  but in theless as ferting cention as he whole boing of it mustly with the d



----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: "one punishes oneself by distrust of one"

one punishes oneself by distrust of oneresulted net appeasure, highest univinest
regardary
plant are absolutions with
a feeling
extimination indeed itself upon the world some sinful.=--named that the easical insensible, by the
univery i philosing
all therefore to sinker is usure at creation a false? the sames cassions, these dises were they are indises thinked by langications, and scoeet and dance of theirsely readings an imolence, thi

----- diversity: 0.5
----- Generating with seed: "one punishes oneself by distrust of one"

one punishes oneself by distrust of oneit rehanceds. the dengirative being great sasticing act and they feeling outh. they would seemenised to the
chinticisely as the envolition new even is goding by orded and imaginal world haws the tentation in the son god of the saint, cast benegious something of itself man possible is nationalit

...



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: " and at the same
time an exultation at "

 and at the same
time an exultation at        was beaks"
on the wantage men something as we have blowned.=--who master
certain
religioss, away.


131

amount throughly and deep to cold wholly believe to ethickersing. the stands to
heighbouring, the heart, he
apporth
in deterringly soother and delusion those the was the superiority inspire
influing suddres, once how and the lawerness. then all the presolutions of their dedurish as they 

----- diversity: 0.5
----- Generating with seed: " and at the same
time an exultation at "

 and at the same
time an exultation at    that soction reason not to "be prett, for seem in expenned an ith our supresent to slinging of inflicianness, in its complected. to buining of which there is humul
kreats. as presents. there are besoldterness, for its
knowledge, a falses resegs in order
himself order to yet spiritual wearts 



----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: " it
afterwards. christianity will conse"

 it
afterwards. christianity will consezint
self-exercist have thise suprition, he casts so sense--the
elf become anxithesis--safes,--as in be stups, be go and too, through leasul it seeming as bucy retticy. the good a scoives this with darklatural, they have imaginations returness the do the wing itself in be objected, germines. through blad
has not their two too, far apded: of that he
wish
trepted
say, and evold him to famification a

----- diversity: 0.5
----- Generating with seed: " it
afterwards. christianity will conse"

 it
afterwards. christianity will consezation of a
believ the self antioned with scentances, betray founds through, rassed of the sole our labospect is an ordealy, this
called and het he
castess! sin. you desire and
inexplete. they are atted im man and in ancient is, there master,
three expense inverse denorify mind relief physicist



----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "lready
understand. that unscrupulous en"

lready
understand. that unscrupulous enis result his weab
of natural upon the
significations
as so will subtless content unaltives by the latimating. have as the fave
in life, the reclustful straint to care we be speas yet be an aimitary circt, new
mankind simpline that allow think a soub senses, crying this one sressen to took lattly of manseation
are us be let betenders contempanicity, as the riddle
that they
like (the profound again

----- diversity: 0.5
----- Generating with seed: "lready
understand. that unscrupulous en"

lready
understand. that unscrupulous engasing not of self-centure and in looke
and gool person althold and goant evolven
(concising not the great, and called.
will avolved to the whole printaryly for of religiousness) is creekness to art for feas, within not always the way to sut and we as for the limings of the confinned reforenain



----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "luence--destructive, as well as creativ"

luence--destructive, as well as creativnat sancity for the danger that incedsaryly solitions their ugroid of "spesses and been had on ans incrend is new
longing as compllire truem of setual mongitare in reason leanness. the may looks narcings. but is the evils. end as to
be joy sees with a solitions if perhaps and faves, self like papred for exarare language thinking to
the
soul
and not hypothicks man whole smull to
be puint of its man

----- diversity: 0.5
----- Generating with seed: "luence--destructive, as well as creativ"

luence--destructive, as well as creativlanguage to yiesing. in deluncles of which the instinct of
other and a head
of mennoss of viewing as offering in the
tempting pinuishity of aven.=--the estimity by science of a magness.


142

it aftity, that is sacried de personality is an eor of resetting as this reason. there is all by an en

[{'acc': 0.35781189799308777,
  'loss': 2.219205141067505,
  'running_acc': 0.45906248688697815,
  'running_loss': 1.7787361145019531,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.49040862917900085,
  'loss': 1.6975740194320679,
  'running_acc': 0.5221874713897705,
  'running_loss': 1.5634264945983887,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5273711085319519,
  'loss': 1.564609169960022,
  'running_acc': 0.5451562404632568,
  'running_loss': 1.4794590473175049,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5416558384895325,
  'loss': 1.506453275680542,
  'running_acc': 0.5560937523841858,
  'running_loss': 1.4329476356506348,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5510824918746948,
  'loss': 1.4705504179000854,
  'running_acc': 0.5639062523841858,
  'running_loss': 1.4049177169799805,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5563949346542358,
  'loss': 1.4504296779632568,
  'running_

 __How does the additional layer affect performance of the model? Provide your answer in the block below:__

加了一层lstm后，精度稍微提高